<a href="https://colab.research.google.com/github/OmdenaAI/SudanChapter_AnalyzeHealthcareAccessibility/blob/main/04_Data_analysis/Geographic/flattened_ndvi_ndwi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Importing libraries</b>

In [1]:
import ast
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

<b>Reading CSV<b>

In [2]:
# Connect to G. drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Downloading file

import gdown

# Google Drive
file_url = "https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR"

output = "geographic_merged.csv"


gdown.download(file_url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR
From (redirected): https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR&confirm=t&uuid=270a9b0e-49a1-43fe-9ffc-87823209510a
To: /content/geographic_merged.csv
100%|██████████| 66.4M/66.4M [00:02<00:00, 32.4MB/s]


'geographic_merged.csv'

In [4]:

file_path = "/content/geographic_merged.csv"
geo = pd.read_csv(file_path)

<b>Exploring the data</b>

In [5]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   GID_L0                   237 non-null    object
 1   GID_L1                   237 non-null    object
 2   GID_L2                   237 non-null    object
 3   GID_L3                   237 non-null    object
 4   COUNTRY                  237 non-null    object
 5   NAME_L1                  237 non-null    object
 6   NAME_L2                  237 non-null    object
 7   NAME_L3                  237 non-null    object
 8   GEOMETRY_L3              237 non-null    object
 9   NAME_ALTERNATIVES_L1     232 non-null    object
 10  TYPE_L1                  237 non-null    object
 11  HASC_L1                  237 non-null    object
 12  ISO_L1                   135 non-null    object
 13  GEOMETRY_L1              237 non-null    object
 14  TYPE_L2                  237 non-null    o

In [6]:
geo.nunique()

,0
GID_L0,1
GID_L1,18
GID_L2,80
GID_L3,237
COUNTRY,1
NAME_L1,18
NAME_L2,79
NAME_L3,233
GEOMETRY_L3,237
NAME_ALTERNATIVES_L1,17


In [7]:
# Converts each record to python literal from string objects

geo["SENTINEL_2_NDVI_L3"] = geo["SENTINEL_2_NDVI_L3"].apply(lambda x: ast.literal_eval(x))
geo["SENTINEL_2_NDWI_L3"] = geo["SENTINEL_2_NDWI_L3"].apply(lambda x: ast.literal_eval(x))
geo["WEATHER_WEEKLY_L1"] = geo["WEATHER_WEEKLY_L1"].apply(lambda x: ast.literal_eval(x))

In [19]:
# Flattening function

def flatten_data(df, column, region_col): #, geom_col, coordinate, centroids
    records = []
    for idx, row in df.iterrows():
        if isinstance(row[column], dict):
            for key, values in row[column].items():
                for item in values:
                    records.append({
                        'date': pd.to_datetime(item['interval_from']),
                        'value': item['ndvi_mean'],
                        'state': row[region_col],
                        # 'geometry': row[geom_col],
                        # 'coordinate': row[coordinate],
                        # 'centroids': row[centroids]
                    })
    return pd.DataFrame(records)

In [20]:
# Flattening relevant columns

ndvi_data = flatten_data(geo, 'SENTINEL_2_NDVI_L3', 'NAME_L1')  # , 'GEOMETRY_L1', 'COORDINATES_L1', 'CENTROID_COORDINATES_L1'
ndwi_data = flatten_data(geo, 'SENTINEL_2_NDWI_L3', 'NAME_L1')  # , 'GEOMETRY_L1', 'COORDINATES_L1', 'CENTROID_COORDINATES_L1'

#weather_data = flatten_data(geo, 'WEATHER_WEEKLY_L1', 'NAME_L1', 'GEOMETRY_L1', 'COORDINATES_L1', 'CENTROID_COORDINATES_L1')

In [21]:
ndvi_data.head()

,date,value,state
0,2015-10-20,0.005310,AlJazirah
1,2015-11-29,0.017227,AlJazirah
2,2015-12-09,0.058856,AlJazirah
3,2015-12-19,0.030519,AlJazirah
4,2015-12-29,0.084125,AlJazirah


In [22]:
ndwi_data.head()

,date,value,state
0,2015-10-20,-0.011545,AlJazirah
1,2015-11-29,-0.040840,AlJazirah
2,2015-12-09,-0.094806,AlJazirah
3,2015-12-19,0.015846,AlJazirah
4,2015-12-29,-0.074044,AlJazirah


In [23]:
# Saving the output in a CSV file

ndvi_data.to_csv('ndvi_data.csv', index=False)
ndwi_data.to_csv('ndwi_data.csv', index=False)

In [24]:
ndvi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141963 entries, 0 to 141962
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    141963 non-null  datetime64[ns]
 1   value   141963 non-null  float64       
 2   state   141963 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 3.2+ MB
